# NLP Homework 5
## Chunking
### Julia Layne

In [1]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

import nltk
import re
from nltk.corpus import stopwords
from string import punctuation

from nltk.metrics import distance
import numpy as np
import pandas as pd

import pprint
import re

### 1.	Compile a list of static links (permalinks) to individual user movie reviews from one particular website. This will be your working dataset for this assignment, as well as for assignments 7 and 8, which together will make up your semester project.   
 - a.	It does not matter if you use a crawler or if you manually collect the links, but you will need at least 100 movie review links. Note that, as of this writing, the robots.txt file of IMDB.com allows the crawling of user reviews.
 - b.	Each link should be to a web page that has only one user review of only one movie, e.g., the user review permalinks on the IMDB site.
 - c.	Choose reviews of movies that are all in the same genre, e.g., sci-fi, mystery, romance, superhero, etc.  
 - d.	Make sure your collection includes reviews of several movies in your chosen genre and that it includes a mix of negative and positive reviews.

In [2]:
reviews = pd.read_csv("reviews.csv")
reviews.URL[0]

'https://www.imdb.com/review/rw6540019/?ref_=tt_urv'

In [3]:
reviews["SOUP"] = '-'
reviews["CONTENT"] = '-'
reviews.head()

URL SOUP CONTENT
0  https://www.imdb.com/review/rw6540019/?ref_=tt...    -       -
1  https://www.imdb.com/review/rw6562703/?ref_=tt...    -       -
2  https://www.imdb.com/review/rw6650913/?ref_=tt...    -       -
3  https://www.imdb.com/review/rw6603814/?ref_=tt...    -       -
4  https://www.imdb.com/review/rw6479388/?ref_=tt...    -       -

In [4]:
reviews.iloc[4, 1]

'-'

In [5]:
import urllib.request

fp = urllib.request.urlopen('https://www.imdb.com/review/rw6540019/?ref_=tt_urv')
mybytes = fp.read()

html_doc = mybytes.decode("utf8")
fp.close()

#print(html_doc)

### 2.	 Extract noun phrase (NP) chunks from your reviews using the following procedure:
 - a.	In Python, use BeautifulSoup to grab the main review text from each link.  
 - b.	Next run each review text through a tokenizer, and then try to NP-chunk it with a shallow parser. 
 - c.	You probably will have too many unknown words, owing to proper names of characters, actors, and so on that are not in your working dictionary. Make sure the main names that are relevant to the movies in your collection of reviews are added to the working lexicon, and then run the NP chunker again.


In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')

print(soup.prettify())
soup.find("div", class_="text show-more__control")

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="app-id=342792525, app-argument=imdb:///review/rw6540019?src=mdot" name="apple-itunes-app"/>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   evansjoshyr's Review of WandaVision - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   if (typeof uex == 'function') {
      uex("ld", "

<div class="text show-more__control">It might have been an error on Disney's part to release only the first 2 episodes at once instead of 4 episodes. While I am on of the few people who absolutely loved the intrigue and mystery of the first three episodes, this exceptional show has been getting a lot of hate due to a lack of plot.<br/><br/>...
And then the 4th episode came out. Not only was it an exceptional episode, but it justified the use of the first three and even gave some answers to the many questions of Wanda vision.<br/><br/>Overall a brilliant show that shouldn't be judged on the first three episodes (especially if your only complaint is a lack of plot)</div>

In [7]:
soup.find("div", class_="text show-more__control").dtype

In [8]:

for i in range(len(reviews.URL)):
    fp = urllib.request.urlopen(reviews.URL[i])
    mybytes = fp.read()
    html_doc = mybytes.decode("utf8")
    fp.close()
    soup = BeautifulSoup(html_doc, 'html.parser')
    reviews.iloc[i, 1] = soup
    found = soup.find("div", class_="text show-more__control")
    soup_review = BeautifulSoup(str(found), 'html.parser')
    reviews.iloc[i, 2] = str(soup_review.get_text())
    
reviews.head()

URL  \
0  https://www.imdb.com/review/rw6540019/?ref_=tt...   
1  https://www.imdb.com/review/rw6562703/?ref_=tt...   
2  https://www.imdb.com/review/rw6650913/?ref_=tt...   
3  https://www.imdb.com/review/rw6603814/?ref_=tt...   
4  https://www.imdb.com/review/rw6479388/?ref_=tt...   

                                                SOUP  \
0  [\n, html, \n, [\n, [\n, <meta charset="utf-8"...   
1  [\n, html, \n, [\n, [\n, <meta charset="utf-8"...   
2  [\n, html, \n, [\n, [\n, <meta charset="utf-8"...   
3  [\n, html, \n, [\n, [\n, <meta charset="utf-8"...   
4  [\n, html, \n, [\n, [\n, <meta charset="utf-8"...   

                                             CONTENT  
0  It might have been an error on Disney's part t...  
1  Writing this with bated breath waiting on epis...  
2  One might wonder that the above adjectives do ...  
3  I can't speak for the reviews that bash this s...  
4  What a waste $25 Million Dollars A episode don...

In [9]:
reviews["NP_CHUNK"] = '-'

In [10]:
#!pip install pattern3

In [19]:
from pattern.en import conjugate, lemma, lexeme, parse,parsetree
from __future__ import print_function
def np_list_parse(doc):
    sentences = nltk.sent_tokenize(doc)
    np_list = []
    #print(sentences)
    for sentence in sentences:
        #print(sentence)
        tree = parse(sentence,
            tokenize = True,  # Tokenize the input, i.e. split punctuation from words.
            tags = True,  # Find part-of-speech tags.
            chunks = True,  # Find chunk tags, e.g. "the black cat" = NP = noun phrase.
            relations = True,  # Find relations between chunks.
            lemmata = True,  # Find word lemmata.
            light = False)
        split_tree = tree.split()
        for phrase in split_tree[0]:
            if 'NP' in phrase[2]:
                np_list.append(phrase[0])
    return np_list

In [20]:
np_list_parse(reviews.iloc[0, 2])

['It',
 'an',
 'error',
 'Disney',
 'part',
 'episodes',
 'episodes',
 'I',
 'the',
 'few',
 'people',
 'the',
 'intrigue',
 'mystery',
 'three',
 'episodes',
 'this',
 'exceptional',
 'show',
 'a',
 'lot',
 'hate',
 'a',
 'lack',
 'plot',
 'it',
 'an',
 'exceptional',
 'episode',
 'it',
 'the',
 'use',
 'some',
 'answers',
 'the',
 'many',
 'questions',
 'Wanda',
 'vision.Overall',
 'a',
 'brilliant',
 'show',
 'three',
 'episodes',
 'your',
 'only',
 'complaint',
 'a',
 'lack',
 'plot']

In [21]:
for i in range(len(reviews.URL)):
    reviews.iloc[i,3] = np_list_parse(reviews.iloc[i, 2])
reviews.NP_CHUNK[0]

['It',
 'an',
 'error',
 'Disney',
 'part',
 'episodes',
 'episodes',
 'I',
 'the',
 'few',
 'people',
 'the',
 'intrigue',
 'mystery',
 'three',
 'episodes',
 'this',
 'exceptional',
 'show',
 'a',
 'lot',
 'hate',
 'a',
 'lack',
 'plot',
 'it',
 'an',
 'exceptional',
 'episode',
 'it',
 'the',
 'use',
 'some',
 'answers',
 'the',
 'many',
 'questions',
 'Wanda',
 'vision.Overall',
 'a',
 'brilliant',
 'show',
 'three',
 'episodes',
 'your',
 'only',
 'complaint',
 'a',
 'lack',
 'plot']

### 3.	Output all the chunks in a single list for each review, and submit that output for this assignment. Also submit a brief written summary of what you did (describe your selection of genre, your source of reviews, how many you collected, and by what means).

In [22]:
parsed_reviews = reviews.copy()
parsed_reviews = parsed_reviews.drop(['URL', 'SOUP'], axis=1)
parsed_reviews.head()

CONTENT  \
0  It might have been an error on Disney's part t...   
1  Writing this with bated breath waiting on epis...   
2  One might wonder that the above adjectives do ...   
3  I can't speak for the reviews that bash this s...   
4  What a waste $25 Million Dollars A episode don...   

                                            NP_CHUNK  
0  [It, an, error, Disney, part, episodes, episod...  
1  [bated, breath, episode, The, whole, premise, ...  
2  [the, above, adjectives, the, Marvel, Cinemati...  
3  [I, the, reviews, this, show, three, episodes,...  
4  [a, waste, 25, Million, Dollars, A, episode, t...

I manually collected permalinks from imdb from the Comedy genre. For about 10 movies or shows I selected roughly even amounts of positive revews (7+ stars) and negative reviews (3 or lower). This was to have a clear set of 'positive' and 'negative' reviews for the sake of classification later.

The urllib package allowed me to rotate through the urls collected. From looking through the HTML, the div with a class of 'text show-more__control' happened to have the review text in it. From that text, using BeautifulSoup, I was able to pull out that single div's text into the CONTENT column of the dataframe. 

Then using NLTK to tokenize these reviews into sentences, the pattern package's parse marked each word by POS and which phrase it belongs to.

In [23]:
reviews.to_csv(r'parsed_reviews.csv', index = False)